In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pickle


# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [2]:
ak = pd.read_csv('../datasets/alaska_single_engine_clean.csv', low_memory=False)

In [3]:
ak.shape

(5659, 31)

In [4]:
ak['has_injury'] = ak.highest_injury_level.apply(lambda x: 0 if x == 'None Reported' else 1)

In [5]:
ak = ak.drop(['ntsb_no', 'mkey', 'n', 'probable_cause', 'far', 'operator', 'event_time', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'air_craft_damage', 'event_type', 'report_type', 'has_safety_rec', 'highest_injury_level', 'event_year'], axis=1)

In [6]:
ak.has_injury.value_counts()

has_injury
0    4044
1    1615
Name: count, dtype: int64

In [7]:
categorical_cols = ak.select_dtypes(exclude=['number']).columns.tolist()

In [8]:
categorical_cols

['city',
 'make',
 'model',
 'air_craft_category',
 'airport_id',
 'airport_name',
 'scheduled',
 'purpose_of_flight',
 'weather_condition',
 'event_season']

In [9]:
ak.columns

Index(['city', 'latitude', 'longitude', 'make', 'model', 'air_craft_category',
       'airport_id', 'airport_name', 'amateur_built', 'scheduled',
       'purpose_of_flight', 'weather_condition', 'event_month', 'event_day',
       'event_season', 'has_injury'],
      dtype='object')

In [10]:
ak_dummies = pd.get_dummies(ak, columns = categorical_cols, drop_first=True)

In [11]:
X = ak_dummies.drop('has_injury', axis=1)
y = ak_dummies['has_injury']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.075, random_state=21)

### Baseline

In [13]:
1 - y.mean()

0.7146138893797491

### Benchmark Logistic Regression Model

In [14]:
pipe_log_reg_bench = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_bench', LogisticRegression(max_iter=1000))
])

In [15]:
pipe_log_reg_bench.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('log_reg_bench', LogisticRegression(max_iter=1000))])

In [16]:
pipe_log_reg_bench.score(X_train, y_train)

0.9073366450133741

In [17]:
pipe_log_reg_bench.score(X_test, y_test)

0.6847058823529412

### Tuned Logistic Regression Model

In [18]:
pipe_log_reg_tuned = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_tuned', LogisticRegression())
])

In [19]:
pipe_log_reg_tuned.get_params()

{'memory': None,
 'steps': [('sc', StandardScaler()), ('log_reg_tuned', LogisticRegression())],
 'verbose': False,
 'sc': StandardScaler(),
 'log_reg_tuned': LogisticRegression(),
 'sc__copy': True,
 'sc__with_mean': True,
 'sc__with_std': True,
 'log_reg_tuned__C': 1.0,
 'log_reg_tuned__class_weight': None,
 'log_reg_tuned__dual': False,
 'log_reg_tuned__fit_intercept': True,
 'log_reg_tuned__intercept_scaling': 1,
 'log_reg_tuned__l1_ratio': None,
 'log_reg_tuned__max_iter': 100,
 'log_reg_tuned__multi_class': 'auto',
 'log_reg_tuned__n_jobs': None,
 'log_reg_tuned__penalty': 'l2',
 'log_reg_tuned__random_state': None,
 'log_reg_tuned__solver': 'lbfgs',
 'log_reg_tuned__tol': 0.0001,
 'log_reg_tuned__verbose': 0,
 'log_reg_tuned__warm_start': False}

In [20]:
pipe_log_reg_tuned_params = {
    'log_reg_tuned__max_iter': [1_000],
    'log_reg_tuned__C': [.001, .01],
    'log_reg_tuned__penalty': ['l1', 'l2']
}

In [21]:
gs_log_reg_tuned = GridSearchCV(pipe_log_reg_tuned,
                                param_grid = pipe_log_reg_tuned_params,
                                cv = 5,
                               n_jobs=6)

In [22]:
%%time
gs_log_reg_tuned.fit(X_train, y_train)

/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 420, 

CPU times: user 2.76 s, sys: 803 ms, total: 3.56 s
Wall time: 13.4 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('sc', StandardScaler()),
                                       ('log_reg_tuned',
                                        LogisticRegression())]),
             n_jobs=6,
             param_grid={'log_reg_tuned__C': [0.001, 0.01],
                         'log_reg_tuned__max_iter': [1000],
                         'log_reg_tuned__penalty': ['l1', 'l2']})

In [23]:
gs_log_reg_tuned.best_params_

{'log_reg_tuned__C': 0.001,
 'log_reg_tuned__max_iter': 1000,
 'log_reg_tuned__penalty': 'l2'}

In [24]:
gs_log_reg_tuned.score(X_train, y_train)

0.8706534199465036

In [25]:
gs_log_reg_tuned.score(X_test, y_test)

0.7364705882352941

In [26]:
y_pred = gs_log_reg_tuned.predict(X_test)

In [27]:
confusion_matrix(y_test, y_pred)

array([[289,  21],
       [ 91,  24]])

In [28]:
print(classification_report(y_test, y_pred, digits = 5))

              precision    recall  f1-score   support

           0    0.76053   0.93226   0.83768       310
           1    0.53333   0.20870   0.30000       115

    accuracy                        0.73647       425
   macro avg    0.64693   0.57048   0.56884       425
weighted avg    0.69905   0.73647   0.69219       425



### Coefficients

In [105]:
# coefficients = np.exp(gs_log_reg_tuned.best_estimator_.named_steps['log_reg_tuned'].coef_[0]*.1)

In [110]:
coefficients = gs_log_reg_tuned.best_estimator_.named_steps['log_reg_tuned'].coef_[0]

In [111]:
feature_names = list(X_train.columns)

In [112]:
coefficients_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefficients})

In [115]:
pd.set_option('display.max_rows', None)
coefficients_df.head(25)

,feature,coefficient
0,latitude,-0.017773
1,longitude,0.017297
2,amateur_built,0.027261
3,event_month,0.009724
4,event_day,0.012611
5,city_100MI S.KNG SLM,-0.012404
6,city_11NM EAST OF SI,0.021879
7,city_18NM ESE KETCHI,-0.005551
8,city_20 ESE NORTH PO,-0.009956
9,city_27 E SAGWON,-0.006885


In [114]:
coefficients_df.sort_values(by='coefficient', ascending=False).head(30)

,feature,coefficient
4219,airport_name_Unknown,0.071593
3140,airport_id_Unknown,0.052266
924,city_SLEETMUTE,0.051643
479,city_KENAI,0.050632
4322,purpose_of_flight_UNK,0.050447
470,city_KASILOF,0.048017
239,city_Chugiak,0.047760
4326,weather_condition_Unknown,0.044177
3917,airport_name_O'MALLEY,0.040544
551,city_Koyukuk,0.039679


In [88]:
coefficients_df.sort_values(by='coefficient', ascending=False).tail(30)

,feature,coefficient
919,city_SKWENTNA,-0.025097
723,city_NENANA,-0.025481
758,city_Nenana,-0.025534
2464,model_R-22,-0.025747
624,city_MONTAGUE ISLAND,-0.025784
3963,airport_name_PRIVATE STRIP,-0.025860
1158,make_Aerospatiale,-0.025933
2256,model_M-5-210C,-0.026106
4308,purpose_of_flight_PERS,-0.027020
1506,model_108-1,-0.027229


In [90]:
coefficients_df.to_csv('../datasets/nolan_log_reg_coef.csv', index=False)

### Pickle

In [31]:
with open ('../pickles/nolan_log_reg.pkl', 'wb') as f:
    pickle.dump('gs_log_reg_tuned.pkl', f)